## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
WORKING_DIR = "/content/drive/MyDrive/asiCD-dev"

%cd drive/MyDrive/asiCD-dev

In [ ]:

!rm -rf ../tmp/

In [ ]:
%ls -a

## Installing Dependencies

In [ ]:
# !pip install tensorflow-gpu==2.1.0
# !pip install tqdm
# !pip install imgaug

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

import random
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image

from tqdm import tqdm_notebook

%matplotlib inline
print(tf.__version__)

run_flag = 0

In [ ]:
from asiCD import model_utils
from asiCD.models import model_vgg16
from asiCD.models import model_resnet18
from asiCD.models import model_densenet21

In [ ]:
config = utilities.read_json("config.json")
print(config)

dataset_path = config["dataset_path"]
AUG_CONFIG = config["aug_config"]

In [ ]:
print(dataset_path)

##  Preprocessing Data
- Required file structure (for ImageDataGenerator)
```
dataset
├───test
│   └───img
├───test_labels
│   └───img
├───train
│   └───img
├───train_labels
│   └───img
├───val
│   └───img
├───val_lables
│   └───img
```
- Ops:
  1. Downsamling
  2. Gaussian Filter (Pending)
  3. Undistorting (Pending)
- Saving processed Images and Masks

### Output File Structure

In [ ]:
%ls

In [ ]:
# Creating output file structure
utilities.create_output_dir()
dataset_path = utilities.create_dataset_dir("output")
run_flag += 1
print(dataset_path)

### Preprocessing

## Loading Images and Labels from Google Drive

In [ ]:
dataset_path = "dataset/swimseg-2-modded"

In [ ]:
train_generator = model_utils.get_img_mask_generators(dataset_path + "/train", aug_config = AUG_CONFIG)
test_generator = model_utils.get_img_mask_generators(dataset_path + "/test", aug_config = AUG_CONFIG)
val_generator = model_utils.get_img_mask_generators(dataset_path + "/val", aug_config = AUG_CONFIG)

In [ ]:
""" TEST """
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

image_generator = image_datagen.flow_from_directory(dataset_path + "/train",
                                                    target_size= (128,128),
                                                    color_mode="rgb",
                                                    class_mode=None,
                                                    seed=0)
mask_generator = mask_datagen.flow_from_directory(dataset_path + "/train_labels",
                                                  target_size= (128,128),
                                                  color_mode="grayscale",
                                                  class_mode=None,
                                                  seed=0)


In [ ]:
img_arr = image_generator.next()
mask_arr = mask_generator.next()
for i in range(0,1):
    image = img_arr[i]
    mask = mask_arr[i]
    
    plt.subplot(1,2,1)
    plt.title(f"Image: {image.shape}")
    plt.imshow(image / 255)
    plt.xticks([])
    plt.yticks([])
    
    plt.subplot(1,2,2)
    plt.title(f"Mask: {mask.shape}")
    plt.imshow(mask[:,:,0] / 255)
    plt.xticks([])
    plt.yticks([])

    plt.show()

""" END """

In [ ]:
# TRAIN_IMAGES_PATH = "/content/drive/My Drive/asiCD/swimseg-dataset-npy/2020-11-14-12-21-train_images.npy"
# TRAIN_LABELS_PATH = "/content/drive/My Drive/asiCD/swimseg-dataset-npy/2020-11-14-12-21-train_labels.npy"

# train_images = load_npy(TRAIN_IMAGES_PATH)
# train_labels = load_npy(TRAIN_LABELS_PATH)

# TEST_IMAGES_PATH = "/content/drive/My Drive/asiCD/swimseg-dataset-npy/2020-11-14-12-21-test_images.npy"
# TEST_LABELS_PATH = "/content/drive/My Drive/asiCD/swimseg-dataset-npy/2020-11-14-12-21-test_labels.npy"

# test_images = load_npy(TEST_IMAGES_PATH)
# test_labels = load_npy(TEST_LABELS_PATH)

# VAL_IMAGES_PATH = "/content/drive/My Drive/asiCD/swimseg-dataset-npy/2020-11-14-12-21-val_images.npy"
# VAL_LABELS_PATH = "/content/drive/My Drive/asiCD/swimseg-dataset-npy/2020-11-14-12-21-val_labels.npy"

# val_images = load_npy(VAL_IMAGES_PATH)
# val_labels = load_npy(VAL_LABELS_PATH)

In [ ]:
models = [model_vgg16, model_resnet18, model_densenet21]

In [ ]:
model = models[1].MultiExpoNet.build(128, 128, 1, 1, 3, 0.001, 10)

In [ ]:
model.summary()

In [ ]:
EPOCHS = 10
BS = 32

train_num = 861
test_num = 51

steps_per_epoch = train_num // BS
validation_steps = test_num // BS

In [ ]:
hist = model.fit(test_generator, 
                 epochs= EPOCHS,
                 steps_per_epoch= steps_per_epoch, 
                 validation_data= val_generator,
                 validation_steps= validation_steps)

# hist = model.fit(x= img_arr, y =mask_arr, epochs=10)

In [ ]:
img_arr.shape

In [ ]:
mask_arr.shape

In [ ]:
model.save("some_weights.h5")

In [ ]:
for sample_in in range(32):
  plt.figure(figsize=(15,15))
  y_pred = model.predict(np.expand_dims(img_arr[sample_in], axis=0))
  plt.subplot(1,5,1)
  plt.imshow(img_arr[sample_in,...]/255)
  plt.title('Input Image')
  plt.subplot(1,5,2)
  plt.imshow(mask_arr[sample_in,:,:,0])
  plt.title('Ground Truth')
  plt.subplot(1,5,3)
  y_pred += np.abs(np.min(y_pred))
  plt.imshow(y_pred[0,:,:,0] / np.max(y_pred))
  plt.title('Predicted Output')
  plt.subplot(1,5,4)
  y_pred = np.where(y_pred<np.average(y_pred),y_pred,255)
  y_pred = np.where(y_pred>np.average(y_pred),y_pred,0)
  plt.imshow(y_pred[0,:,:,0] / np.max(y_pred))
  plt.title('Output with thresholds')
  plt.savefig(f'./predictions/Image{sample_in}.png', dpi=400, bbox_inches ='tight' )
# plt.subplot(1,5,5)
# cax = plt.axes([0.85, 0.1, 0.075, 0.8])
# plt.colorbar(cax=cax,shrink=0.1) # need to work on positioning. orientation=horizontal is not helpful
plt.show() # plt.show() destorys the instance after display. Save fig before showing.

In [ ]:
np.max(y_pred)

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
# List of output layer from the previously defined model
successive_outputs = [layer.output for layer in model.layers]

# Building a model with 1 input to multiple output structure
visualization_model = tf.keras.models.Model(
    inputs=model.input, outputs=successive_outputs
)

# preparing a random image from the training dataset
train_dir = '/content/drive/MyDrive/asiCD-dev/dataset/swimseg-2-modded/train/img'
train_files_list = os.listdir(train_dir)
img_path = random.choice([os.path.join(train_dir,f) for f in train_files_list])
img = load_img(
    img_path,
    target_size=(128, 128),
)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) # (or) ==> x = x.reshape((1,)+x.shape) 
# print(x.shape)
x /= 255.0

successive_feature_maps = visualization_model.predict(x)

layer_names = [layer.name for layer in model.layers]
# print(layer_names)
for layer_name,feature_map in zip(layer_names,successive_feature_maps):
    if len(feature_map.shape) == 4:
        n_features = feature_map.shape[-1]
        size = feature_map.shape[1]

        display_grid = np.zeros((size, size * n_features))

        for i in range(n_features):
            x = feature_map[:,:,:,i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x,0,255).astype('uint8')
            display_grid[:, i* size : (i+1)*size] = x
    scale = 20./n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid('Off')
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
# running inference on uploaded images

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for filename in uploaded.keys():

  path = './'+filename
  img = image.load_img(path,target_size=(128,128))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  # print(x.shape)
  plt.gcf()
  plt.subplot(1,2,1)
  plt.imshow(x[0,...]/255.0)  
  plt.subplot(1,2,2)
  pred = model.predict(x)
  plt.imshow(pred[0,...],cmap='viridis')  
  plt.savefig('./predictions/pred.png',dpi=400)
  plt.show()